In [ ]:
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install seaborn
!pip install matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Importo dataset di train e test

In [ ]:
# importo train e test dataset
train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")

In [ ]:
# prima vista del dataset, si possono vedere le colonne e alcuni valori contenuti
train_dataset

In [ ]:
# importo le stat dei vari pokemon
pokemon = pd.read_csv("pokemon.csv")

In [ ]:
# analisi del data per conoscere il tipo di dato per colonna, ci viene anche detto quanti sono complessivamente i valori non nulli
# nella colonna
train_dataset.info()

In [ ]:
# analisi del data per conoscere il tipo di dato per colonna, ci vengono forniti anche informazioni sui valori nulli presenti nel dataset
pokemon.info()

In [ ]:
# si vuole capire quanti valori nulli siano effettivamente presenti nel dataset
train_dataset.isna().sum()

In [ ]:
# si vuole capire quanti valori nulli siano effettivamente presenti nel dataset
pokemon.isna().sum()

vi sono molti valori nulli nella colonna "Type 2", in fase di preprocessing si dovrà valutare come trattare questi valori

# Preprocessing

In [ ]:
# join del dataset di train contenente solo i risultati dei combattimenti con il dataset delle statistiche, prima per il primo
# pokemon che attacca e poi per il secondo
# il valore sul quale viene fatto il join è l'identificativo nel pokedex del pokemon
train_dataset = train_dataset.join(pokemon.set_index("#"), on="First_pokemon")
train_dataset = train_dataset.join(pokemon.set_index("#"), on="Second_pokemon", rsuffix="_second")

In [ ]:
# osserviamo come si modifica il dataset a seguito del join
pd.set_option("display.max_columns", None)
train_dataset.head()

In [ ]:
# ripeto tutto anche per il test dataset
test_dataset = test_dataset.join(pokemon.set_index("#"), on="First_pokemon")
test_dataset = test_dataset.join(pokemon.set_index("#"), on="Second_pokemon", rsuffix="_second")
test_dataset.head()

In [ ]:
# controllo quanti siano i valori distinti per name, voglio vedere se sono univoci, in caso posso droppare la colonna perché non
# influisce sul risultato finale
pokemon['Name'].nunique()

su una tabella di 800 righe 799 contengono valori unici, l'analisi ha dato il risultato sperato

In [ ]:
# il nome dei pokemon non è rilevante essendo unico per ogni pokemon, quindi si può droppare la colonna
# sia il nome dei primo che del secondo pokemon
train_dataset = train_dataset.drop('Name', axis=1)
train_dataset = train_dataset.drop('Name_second', axis=1)

#ripeto tutto per il dataset di test
test_dataset = test_dataset.drop('Name', axis=1)
test_dataset = test_dataset.drop('Name_second', axis=1)

In [ ]:
# analisi del data per conoscere il tipo di dato per colonna e i valori nulli
test_dataset.info()

In [ ]:
# analisi del data per conoscere il tipo di dato per colonna e i valori nulli
train_dataset.info()

Ci sono molte righe che presentano null come valore per "Type 2"

Quando il tipo 2 è nullo lo sostituisco con il tipo 1, perché (tenendo in considerazione il gioco) avere un pokemon con due tipi uguali è come avere un pokemon con un solo tipo, per esempio un pokemon "type 1"= foglia e "type 2"=foglia è come avere un pokemon che ha "type 1"= foglia e "type 2"=null, evito di droppare la colonna "type 2" perché preferisco costruire prima la heatmap e vedere la correlazione, in caso poi la droppo

Un altro approccio poteva essere quello di droppare direttamente la colonna "Type 2" data l'elevata presenza di valori nulli, io ho preferito aspettare la creazione della heatmap per conoscere la correlazione prima di droppare la colonna

In [ ]:
# modifica dei valori nulli nella colonna "Type 2", il valore nullo viene sostituito con il valore del "Type 1"

train_dataset['Type 2'].fillna(train_dataset['Type 1'],inplace=True)
train_dataset['Type 2_second'].fillna(train_dataset['Type 1_second'],inplace=True)


test_dataset['Type 2'].fillna(test_dataset['Type 1'],inplace=True)
test_dataset['Type 2_second'].fillna(test_dataset['Type 1_second'],inplace=True)

In [ ]:
# i valori delle righe per la colonna "type 2" sono state correttamente modificate, non sono più presenti valori nulli
train_dataset.info()

## Modifico i valori della colonna "Winner", invece di riportare l'identificativo del pokemon vincente inserisco il valore "1" se ha vinto il primo pokemon e "0" se invece ha vinto il secondo pokemon
### Legenda: 
### 1 -> ha vinto il primo
### 0 -> ha vinto il secondo

In [ ]:
# ogni volta che il vincitore è il primo pokemon pongo campo "winner" pari a 1, 0 altrimenti
for i in range(train_dataset.shape[0]):
    if train_dataset.Winner[i] ==  train_dataset.First_pokemon[i]:
        train_dataset.Winner[i] = 1
    else:
        train_dataset.Winner[i] = 0
        
# ripeto lo stesso anche per il test dataset

for i in range(test_dataset.shape[0]):
    if test_dataset.Winner[i] ==  test_dataset.First_pokemon[i]:
        test_dataset.Winner[i] = 1
    else:
        test_dataset.Winner[i] = 0

In [ ]:
# controllo che effettivamente sia avvenuta la modifica
test_dataset.head()

In [ ]:
# dato che in "Winner" è già contenuta l'informazione relativa alla vittoria del primo o del secondo pokemon, il valore contenuto
# in "First_pokemon" e in "Second_pokemon" non è più utile, quindi può essere droppato
train_dataset = train_dataset.drop('First_pokemon', axis=1)
train_dataset = train_dataset.drop('Second_pokemon', axis=1)

#ripeto tutto per il dataset di test
test_dataset = test_dataset.drop('First_pokemon', axis=1)
test_dataset = test_dataset.drop('Second_pokemon', axis=1)

In [ ]:
# le due colonne sono state correttamente droppate
test_dataset.head()

In [ ]:
# le due colonne sono state correttamente droppate
train_dataset.head()

# Encoding

Nella fase di encoding devo trasformare i valori non numerici in valori numerici

Creo due funzioni che mi restituiscono rispettivamente i nomi delle colonne di tipo object e bool (tutte le colonne non numeriche) per semplificare l'encoding

In [ ]:
# funzione che restituisce i nomi di tutte le colonne di tipo object
def get_categorical_columns(df):
    return [column for column in df.columns if df.dtypes[column] == 'object']

# funzione che restituisce i nomi di tutte le colonne di tipo bool
def get_bool_columns(df):
    return [column for column in df.columns if df.dtypes[column] == 'bool']

In [ ]:
# lista che contiene tutte le colonne con valori di tipo object

categorical_features = get_categorical_columns(train_dataset)

# lista che contiene tutte le colonne con valori di tipo bool

binary_features = get_bool_columns(train_dataset)

Utilizzo il label encoder per tutte le features perché come classificatore utilizzerò Decision Tree e Random Forest, se avessi utilizzato la Logistic Regression avrei dovuto utilizzare OneHot per le features non binarie

In [ ]:
# encoding per features nel test e train dataset
labelEncoder_X = LabelEncoder()

for element in categorical_features:
    train_dataset[element] = labelEncoder_X.fit_transform(train_dataset[element])

for element in categorical_features:
    test_dataset[element] = labelEncoder_X.fit_transform(test_dataset[element])

for element in binary_features:
    train_dataset[element] = labelEncoder_X.fit_transform(train_dataset[element])

for element in binary_features:
    test_dataset[element] = labelEncoder_X.fit_transform(test_dataset[element])

# Features Engineering

Una volta effettuato l'encoding dei valori è possibile tracciare grafici e generare una heatmap per procedere con un'analisi più approfondita delle features, al fine di individuare le features rilevanti e non rilevanti cosi da droppare quelle che costituiscono rumore e migliorare il risultato della classificazione

In [ ]:
# vista del dataset a seguito dell'encoding
test_dataset.head()

In [ ]:
# vista del dataset a seguito dell'encoding
train_dataset.head()

## Heatmap

Una volta fatto l'encoding vado a creare l'heatmap per conoscere le correlazioni tra le diverse features e capire quali features sia possibile droppare, in particolare cerco features che abbiano correlazione rasente lo zero con la colonna "Winner" o molto alta con un'altra features (esclusa ovviamente Winner)

In [ ]:
# creazione della heatmap

plt.figure(figsize=(25,10))
correlation = train_dataset.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

le features che hanno correlazione maggiore con "Winner" sono:
- HP
- Attack
- Sp. Atk
- Sp. Def
- Speed
- Legendary
- (le stesse features ma per il secondo pokemon)

le features che hanno minore correlazione con "Winner" sono:
- Type 1
- Type 2
- Defense
- Generation
- (le stesse features ma per il secondo pokemon)

queste ultime sono le features sulle quali concentreremo l'analisi per capire se vadano droppate o meno

## Generation

La generazione non ha un'alta correlazione con il risultato del combattimento, che in effetti ha senso come cosa perché i pokemon di generazione precedenti/successive non dovrebbero essere più o meno prestanti durante un combattimento (però questo essendo biased dal videogioco)

Oltre all'analisi dei risultati delle heatmap vado anche a tracciare i grafici relativi alla generation per capire se vi siano altre evidenze oltre alla bassa correlazione che possano giustificare l'eliminazione di questa colonna dal dataset

In [ ]:
# il grafico mostra che la distribuzione dell'attributo generation è abbastanza uniforme, a conferma che la sua eliminazione
# sia corretta (non c'è una generazione il cui numero di vittorie e sconfitte sia particolarmente sbilanciato)
sns.countplot(x="Generation", hue="Winner", data=train_dataset)
plt.show()

In [ ]:
# il grafico mostra che la distribuzione dell'attributo generation è abbastanza uniforme, a conferma che la sua eliminazione
# sia corretta (non c'è una generazione il cui numero di vittorie e sconfitte sia particolarmente sbilanciato)
sns.countplot(x="Generation_second", hue="Winner", data=train_dataset)
plt.show()

In [ ]:
# la correlazione tra 'Winner' e 'Generation' è bassa e la distribuzione di "Generation" è uniforme per cui si può droppare
# la colonna
train_dataset = train_dataset.drop('Generation', axis=1)
train_dataset = train_dataset.drop('Generation_second', axis=1)

#ripeto tutto per il dataset di test
test_dataset = test_dataset.drop('Generation', axis=1)
test_dataset = test_dataset.drop('Generation_second', axis=1)

## Type

Nel videogioco il type influenza molto l'esito di una battaglia, però nel dataset non si fa per esempio riferimento alla mosse utilizzate (vero motivo per cui il type dovrebbe influenzare) quindi è utile andare ad analizzare nel dettaglio questa feature (per evitare di essere biased dal gioco)

La heatmap, data la bassa correlazione, indica che si può droppare la feature, un'analisi con i grafici (distribuzioni) confermerà questa ipotesi

In [ ]:
# la distribuzione è abbastanza uniforme, c'è giusto un picco tra il 15-esimo e il 20-esimo tipo, però si può considerare
# poco significativa come caratteristica per determinare il vincitore in una battaglia
ax = sns.displot(x="Type 1", hue="Winner", data=train_dataset, kind="kde")
plt.xlabel('Type 1')
plt.ylabel('Density ')
plt.title("Total Pokemon by Type 1")

In [ ]:
# valgono le stesse considerazioni fatte per il grafico precedente, anche se qua il picco sta tra il decimo e il quindicesimo tipo
ax = sns.displot(x="Type 1_second", hue="Winner", data=train_dataset, kind="kde")
plt.xlabel('Type 1_second')
plt.ylabel('Density ')
plt.title("Total Pokemon by Type 1_second")

Non tracciamo anche i grafici per il Type 2, la correlazione del Type 2 con il Type 1 è particolarmente alta (vedi heatmap) per cui sarebbe stata comunque droppata perché basta solo una delle due caratteristiche per addestrare il modello entrambe sono superflue, ma abbiamo visto come anche Type 1 non sia rilevante

Vengono quindi droppate entrambe

In [ ]:
# elimino le features relative al type dal dataset

train_dataset = train_dataset.drop('Type 1', axis=1)
train_dataset = train_dataset.drop('Type 1_second', axis=1)

train_dataset = train_dataset.drop('Type 2', axis=1)
train_dataset = train_dataset.drop('Type 2_second', axis=1)

#ripeto tutto per il dataset di test
test_dataset = test_dataset.drop('Type 1', axis=1)
test_dataset = test_dataset.drop('Type 1_second', axis=1)

test_dataset = test_dataset.drop('Type 2', axis=1)
test_dataset = test_dataset.drop('Type 2_second', axis=1)

# Statistiche dei pokemon

E' interessante analizzare la differenza dei valori delle carattistiche dei pokemon che si scontrano in un combattimento, in questo modo si riesce a vedere in un solo grafico come i valori delle statistiche dei due pokemon influenzino il risultato finale del combattimento

Da questo punto di vista un combattimento tra due pokemon che hanno rispettivamente att_1 = 5 e att_2 = 7 è simile ad un combattimento con due pokemon che hanno rispettivamente att_1 = 55 e att_2 = 57, così dovrebbe essere evidenziata l'importanza delle varie statistiche rispetto al risultato finale (in questo esempio è stato analizzato solo l'attacco, però lo stesso discorso vale per tutte le statistiche del pokemon)

In [ ]:
# creaiamo una nuova colonna (1 per ogni differenza di valori) rappresentante la differenza dei valori degli attributi dei due pokemon
train_dataset["Differenza_attacco"] = train_dataset.Attack - train_dataset.Attack_second
train_dataset["Differenza_difesa"] = train_dataset.Defense - train_dataset.Defense_second
train_dataset["Differenza_speed"] = train_dataset.Speed - train_dataset.Speed_second
train_dataset["Differenza_attacco_special"] = train_dataset["Sp. Atk"] - train_dataset["Sp. Atk_second"]
train_dataset["Differenza_difesa_special"] = train_dataset["Sp. Def"] - train_dataset["Sp. Def_second"]
train_dataset["Differenza_Hp"] = train_dataset.HP - train_dataset.HP_second

In [ ]:
# stesso lavoro per il datest di test 
test_dataset["Differenza_attacco"] = test_dataset.Attack - test_dataset.Attack_second
test_dataset["Differenza_difesa"] = test_dataset.Defense - test_dataset.Defense_second
test_dataset["Differenza_speed"] = test_dataset.Speed - test_dataset.Speed_second
test_dataset["Differenza_attacco_special"] = test_dataset["Sp. Atk"] - test_dataset["Sp. Atk_second"]
test_dataset["Differenza_difesa_special"] = test_dataset["Sp. Def"] - test_dataset["Sp. Def_second"]
test_dataset["Differenza_Hp"] = test_dataset.HP - test_dataset.HP_second

Tracciamo i grafici relativi alle distribuzioni, sull'asse x abbiamo i valori assunti dalle varie differenze e sull'asse y invece la densità di combattimenti che presentano quel determinato valore (ci sono due curve per ogni grafico, una rappresenta i combattimenti in cui il primo pokemon ha perso e l'altra quelli in cui il primo pokemon ha vinto)

Per vedere se una feature è rilevante dobbiamo avere che le distribuzioni relative a 0 e 1 sono visibilmente distinte (posizionate in posti diversi del grafico) perché ciò vuol dire che un pokemon vince/perde in base ai valori della statistica presa in considerazione

Se invece i due grafici (relativi a 0 e 1) sono quasi sovrapposti vuol dire che a parità di differenza di attacco/difesa/etc. il pokemon ha la stessa probabilità di vincere/perdere una battaglia e quindi quella caratteristica del pokemon non influenza il risultato finale, evidenza che quella feature possa essere droppata

In [ ]:
# andiamo a vedere le distribuzioni per le varie differenze per vedere quali valori effettivamente siano rilevanti per l'analisi
sns.displot(x="Differenza_attacco", hue="Winner", data=train_dataset, kind="kde")
plt.show()
sns.displot(x="Differenza_difesa", hue="Winner", data=train_dataset, kind="kde")
plt.show()
sns.displot(x="Differenza_speed", hue="Winner", data=train_dataset, kind="kde")
plt.show()
sns.displot(x="Differenza_attacco_special", hue="Winner", data=train_dataset, kind="kde")
plt.show()
sns.displot(x="Differenza_difesa_special", hue="Winner", data=train_dataset, kind="kde")
plt.show()
sns.displot(x="Differenza_Hp", hue="Winner", data=train_dataset, kind="kde")
plt.show()

### Defense

Dalle varie distribuzioni si nota immediatamente che le curve realtive a "Differenze_difesa" sono centrate su 0 e le due curve sono quasi coincidenti, ciò vuol dire che la feature non è particolarmente rilevante per la nostra analisi

Già dalla heatmap avevamo notato che i valori di Defense per il primo e il secondo pokemon erano poco correlati con il valore di Winner, quindi sembra abbastanza evidente che la feature non sia rilevante per il training del modello

Tutte le altre caratteristiche hanno curve decisamente distinte, in particolare "Differenza_speed", già nella lettura della heatmap era stato sottolineata l'alta correlazione tra "Winner" e "Speed"

In [ ]:
# per essere certi se droppare la feature sulla difesa andiamo a vedere anche la heatmap relativa al dataset contenente anche
# le differenze

plt.figure(figsize=(25,10))
correlation = train_dataset.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

Dalla heatmap si può notare come la correlazione sia bassa anche per "Differenza_difesa" con "winner", per "Defense" e "Defense_second" l'avevamo già visto prima

Si possono quindi droppare queste colonne

In [ ]:
# droppo la colonna relativa a Defense/Defense_second/Differenza_difesa

train_dataset = train_dataset.drop('Defense', axis=1)
train_dataset = train_dataset.drop('Differenza_difesa', axis=1)
train_dataset = train_dataset.drop('Defense_second', axis=1)

# ripeto tutto anche per il test dataset
test_dataset = test_dataset.drop('Defense', axis=1)
test_dataset = test_dataset.drop('Differenza_difesa', axis=1)
test_dataset = test_dataset.drop('Defense_second', axis=1)

#### L'analisi relativa alla feature che avevano una bassa correlazione con la colonna "Winner" è conclusa e sono state droppate tutte le colonne che non sembravano essere rilevanti per il risultato del combattimento

### Analizzo ora altre due feature: "Legendary" e "Winner"

La prima la analizzo per capire quanto il fatto che un pokemon sia leggendario effettivamente influenzi il risultato finale, nel videogioco i pokemon leggendari sono pokemon rari e molto forti (si potrebbe quindi pensare che l'essere leggendario effettivamente sia un vantaggio per un pokemon in combattimento), oltre al valore della correlazione nella heatmap vado anche a studiare i grafici relativi ai valori di "Legendary" per confermare/confutare l'importanza di questa feature nella classificazione

La feature Winner la analizzo per capire se le vittorie dei combattimenti sono distribuite in maniera abbastanza uniforme tra i due pokemon, sia per capire se cominciare per primo possa portare un vantaggio/svantaggio nel combattimento che per capire se vi sono abbastanza dati per addestrare correttamente il modello a riconoscere entrambe le tipologie di vincitore

## Legendary

Il primo grafico analizza la quantità di vittorie/sconfitte per i pokemon che attaccano per primi distinguendo se questi siano leggendari o meno

In [ ]:
# distribuzione delle vittorie/sconfitte per pokemon leggendari e non
sns.countplot(x="Legendary", hue="Winner", data=train_dataset)
plt.show()

Quando il pokemon non è leggendario la differenza tra i combattimenti persi e quelli vinti non è eccessivamente rilevante, è interessante invece vedere che quando un pokemon è leggendario tende a vincere più combattimenti

Il secondo grafico analizza la quantità di vittorie/sconfitte per i pokemon che attaccano dopo distinguendo se questi siano leggendari o meno

In [ ]:
# distribuzione delle vittorie per pokemon leggendari e non
sns.countplot(x="Legendary_second", hue="Winner", data=train_dataset)
plt.show()

Quando il pokemon non è leggendario la differenza tra i combattimenti persi e quelli vinti non è eccessivamente rilevante, è interessante invece vedere che quando un pokemon è leggendario tende a vincere più combattimenti (stessa considerazione fatta per il primo pokemon)

Potrebbe essere più interessante però vedere in un solo grafico la caratteristica dei due pokemon e il numero di vittorie/sconfitte, per questo creo una nuova colonna nel dataset contenente il valore della differenza tra i valori di "Legendary" tra il primo e il secondo pokemon ("Legendary" - "Legendary_second")

Nello specifico il valore "Diffenza_legendary" assumerà i seguenti valori:
- 1 -> se il primo pokemon è leggendario e il secondo no
- 0 -> se entrambi i pokemon sono leggendari/nessuno dei due è leggendario (assumiamo che se entrambi sono leggendari è come se entrambi non lo fossero)
- -1 -> se il primo pokemon non è leggendario e il secondo si

In [ ]:
# Creo la nuova colonna nel dataset di training
train_dataset['Differenza_legendary'] = train_dataset['Legendary'] - train_dataset['Legendary_second']

# Ripeto lo stesso anche per il dataset di testing
test_dataset['Differenza_legendary'] = test_dataset['Legendary'] - test_dataset['Legendary_second']

Vado adesso a graficare i valori della nuova colonna rispetto al numero di vittorie del primo o del secondo pokemon

In [ ]:
# distribuzione delle vittorie/sconfitte per pokemon leggendari e non (dati relativi sia al primo che al secondo pokemon)
sns.countplot(x="Differenza_legendary", hue="Winner", data=train_dataset)
plt.show()

Anche da questo grafico si rende evidente come l'essere leggendario in effetti avvantaggi un pokemon in battaglia, quindi non conviene droppare la feature perché sembra essere rilevante nell'addestramento del modello

Vado però a generare anche l'heatmap, perché a questo punto "Differenza_legendary" codifica le informazioni presenti in "Legendary" e "Legendary_second", quindi potrebbe essere utile droppare queste due colonne per diminuire la complessità del modello

In [ ]:
# Creazione della heatmap per vedere la correlazione tra "Differenza_legendary/Legendary/Legendary_second"
plt.figure(figsize=(25,10))
correlation = train_dataset.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

Come ipotizzato la correlazione tra "Differenza_legendary" e "Legendary"/"Legendary_second" è molto elevato e la colonna "Differenza_legendary" ha una correlazione maggiore delle altre due con "Winner" quindi possiamo droppare la colonne "Legendary" e "Legendary_second" senza perdite di informazioni rilevanti al fine della classificazione

In [ ]:
# droppo la colonna relativa a "Legendary" e "Legendary_second"

train_dataset = train_dataset.drop('Legendary', axis=1)
train_dataset = train_dataset.drop('Legendary_second', axis=1)


# ripeto tutto anche per il test dataset
test_dataset = test_dataset.drop('Legendary', axis=1)
test_dataset = test_dataset.drop('Legendary_second', axis=1)

## Winner

Procedo con l'analisi dei valori della colonna "Winner", come specificato prima questa analisi è finalizzata a capire se il database sia equilibrato (numero di esempi per le vittorie del primo pokemon e del secondo simili), questo perché teoricamente il classificatore funziona bene se ha visto abbastanza esempi di entrambe le categorie (primo vincitore-secondo vincitore).

Inoltre può essere utile per capire se iniziare per primo possa essere un vantaggio/svantaggio per il pokemon

### 1 = il primo vince il combattimento
### 0 = il secondo vince il combattimento

In [ ]:
# il grafico mostra il numero di esempi che hanno come vincitore rispettivamente il primo e il secondo pokemon
sns.countplot(x="Winner", data=train_dataset)
plt.show()

Il numero di esempi per le due colonne è abbastanza simile, quindi entrambi i casi dovrebbero essere classificati correttamente

Il numero di esempi nei quali il pokemon che muove per secondo vince è maggiore, ciò potrebbe indicare un leggero vantaggio nell'attaccare dopo, però la differenza è veramente minima rispetto al numero totale di esempi

#### Controllo un'ultima volta in dataset di train e test per verificare che effettivamente tutte le modifiche fatte fino ad ora ci siano

In [ ]:
train_dataset

In [ ]:
test_dataset

# Training

In [ ]:
y_train = train_dataset.Winner          
X_train = train_dataset.drop(['Winner'], axis=1)  

y_test = test_dataset.Winner         
X_test = test_dataset.drop(['Winner'], axis=1)

## Decision Tree

Il primo modello utilizzato per la creazione del classificatore è un Decision Tree

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)


y_pred = tree.predict(X_test)
accuracy_lr = tree.score(X_test,y_test)

print(tree)
print('\n')
print("Accuracy: {:.3f}%".format(accuracy_lr*100))

L'accuracy del modello non è male, per avere maggiori informazioni sulla classificazione vado ad analizzare:
- la confusion matrix che ci mostra il numero di falsi positivi/negativi oltre al numero di combattimenti correttamente classificati
- la precision
- la recall
- f1-score

In [ ]:
print('\n')
cm_lr = confusion_matrix(y_test,tree.predict(X_test))
f, ax = plt.subplots(figsize = (5,5))
sns.heatmap(cm_lr, annot = True, linewidths = 0.5, color = "red", fmt = ".0f", ax=ax)
plt.xlabel("y_predicted")
plt.ylabel("y_true")
plt.title("Confusion Matrix of Decision Tree Classifier")
plt.show()
print('\n\n')
print(classification_report(y_test,y_pred))
print('\n\n')

## Random forest

Oltre al Decision Tree utilizzo anche la Random Forest come metodo per la classificazione. Questo modello non è stato spiegato a lezione, ecco come funziona: vengono costruiti una moltitudine di Decision Tree durante il training e il risultato finale dato dal classificatore è il risultato scelto dalla maggior parte dei Decision Tree creati. Questo classificatore dovrebbe migliorare l'accuracy del modello.

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy_lr = clf.score(X_test,y_test)

print(clf)
print('\n')
print("Accuracy: {:.3f}%".format(accuracy_lr*100))

L'accuracy del modello non è male, per avere maggiori informazioni sulla classificazione vado ad analizzare:
- la confusion matrix che ci mostra il numero di falsi positivi/negativi oltre al numero di combattimenti correttamente classificati
- la precision
- la recall
- f1-score

In [ ]:
print('\n')
cm_lr = confusion_matrix(y_test,clf.predict(X_test))
f, ax = plt.subplots(figsize = (5,5))
sns.heatmap(cm_lr, annot = True, linewidths = 0.5, color = "red", fmt = ".0f", ax=ax)
plt.xlabel("y_predicted")
plt.ylabel("y_true")
plt.title("Confusion Matrix of Random Forest Classifier")
plt.show()
print('\n\n')
print(classification_report(y_test,y_pred))
print('\n\n')

La random forest, come ipotizzato, ha prestazioni migliori del Decision Tree